<a href="https://colab.research.google.com/github/MrckHckr/orbit-validation/blob/main/notebooks/validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copia il progetto GitHub su Colab
!git clone https://github.com/MrckHckr/orbit-validation.git


Cloning into 'orbit-validation'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 41 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (41/41), 10.83 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
# Ispeziona il progetto
!apt-get install -y tree
!tree orbit-validation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (89.0 kB/s)
Selecting previously unselected package tree.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
orbit-validation
├── figures
├── notebooks
│   └── validation.ipynb
├── README.md
├── requirements.txt
└── src
    ├── coord_utils.py
    ├── __init__.py
    ├── src
    │   └── time_utils.py
    └── time_utils.py

4 directories, 7 files


In [3]:
# Diciamo a Python dove trovare il codice del repository
import sys
from pathlib import Path

repo_root = Path("/content/orbit-validation")
sys.path.insert(0, str(repo_root))

print("PYTHONPATH OK:", repo_root)



PYTHONPATH OK: /content/orbit-validation


In [4]:
# Guarda cosa c’è dentro il repository
!ls /content/orbit-validation


figures  notebooks  README.md  requirements.txt  src


In [5]:
# Converte una data UTC → TT, serve per validare un propagatore orbitale
from src.time_utils import utc_to_tt

t_tt = utc_to_tt("2025-01-01T12:00:00")
print("TT:", t_tt.iso)
print("JD:", t_tt.jd)


TT: 2025-01-01 12:01:09.184
JD: 2460677.000800741
